In [ ]:
import pandas as pd
import anndata as ad
import scanpy as sc
from arboreto.utils import load_tf_names
from arboreto.algo import grnboost2
import numpy as np

In [ ]:
#Load the data
data = sc.read_h5ad('../../Processed_Data/NetworkData_HVGs_basal_5ht6ho_without_PTPRC_Adgre1.h5ad')#Change path as required.

#Subset the data for 6Ho
ho6_data = data[data.obs['orig.ident'] == '6Ho']

#Load the TFs
tfs = load_tf_names('../../Processed_Data/allTFs_mm.txt')

In [ ]:
#Get the expression data
counts_df_6ho_basal = pd.DataFrame(ho6_data.X, index=ho6_data.obs_names,columns=ho6_data.var_names)

#Get the TFs that are in the data
tfs = sorted(list((set(tfs).intersection(ho6_data.var_names))))

In [ ]:
len(tfs)

# RUN GRNBOOST2 for all TFs but in batches of 10 TFs 

In [ ]:
#Run grnboost2 with SGBM parameters, without a seed - 6ho_basal
for i in range(0,20):#This will run the loop 20 times:
    print(f'Run: {i}')
    #Run GRNBOOST2 for batches of 10 TFs.
    #Create a variable that will contain the entire edge list
    adjacencies = None
    index = 0
    iterations = np.floor(len(tfs)/10)
    n = 10
    count = 0
    #Run in batches of 10
    while count < iterations + 1:
        
        curr_tfs = tfs[index:index+n]
        curr_adjacencies = grnboost2(counts_df_6ho_basal, tf_names=curr_tfs, verbose=False)
        
        if adjacencies is None:
            adjacencies = curr_adjacencies
        else:
            adjacencies = pd.concat([adjacencies, curr_adjacencies], axis=0)
        index += 10
        count += 1
        if count == iterations:
            n = int(len(tfs) - iterations*n)
        print(f'Done for {count*10} TFs')
        ##Save results for every 100 TFs if needed
        # if count%10 == 0:
        #     adjacencies.to_csv(f'../../results/results_step_1_grnboost2_6ho_basal/6Ho_basal_adjacencies_run{i}_{count*10}TFs.csv')
    adjacencies.to_csv(f'../../Results/Results_no_ptprc_adgre1/SCENIC_results/results_step1_6ho_basal/6Ho_basal_adjacencies_run{i}_allTFs.csv')